In [2]:
import cv2
import pandas as pd
import numpy as np
import cascade as cs
from utils import *
from timeit import default_timer as timer

In [42]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [41]:
def Draw_Boxes_on_face(image):
    cascade = create_cascade()
    sart = timer()
    positives = positive_rectangles(image, 19, 1.25, cascade)
    bins = make_partition(positives)
    boxes = get_all_boxes(bins)
    boxed_image = draw_boxes(image, boxes)
    end = timer()
    time_take = end - sart
    return boxed_image, time_take


def create_cascade():
    X_train_fe,y_train = Create_data(1000)
    Strong_Classifiers,No_of_samples_per_layer,No_of_positive_samples_per_layer,No_of_negative_samples_per_layer = np.array(Train_Cascade(X_train_fe, y_train))
    return Strong_Classifiers 



# getting all the possitive windows
def positive_rectangles(image, window_size_base, ratio, cascade):
    sizes = window_sizes(image, window_size_base, ratio)
    rectangles = []
    for size in sizes:
        rectangles += positive_rectangles_size(image, size, window_size_base, cascade)
    return rectangles


def window_sizes(img, min_wind, ratio):
    sizes = []
    temp = min_wind
    while(temp < min(img.shape)):
        sizes.append(temp)
        temp *= ratio
    return sizes


def positive_rectangles_size(image, size, window_size_base, cascade):
    width = image.shape[1] - size + 1
    height = image.shape[0] - size + 1
    dim = (width, height)
    new_image = cv2.resize(image, dim)

    Ans = []
    for i in range(height):
        for j in range(width):
            window = new_image[i:i+window_size_base, j:j+window_size_base]
            pos = cs.Cascade_Classifier_predict_single_Img(window, 1, cascade)
            if pos == True:
                rect = [Point(j, i), Point(j + window_size_base, i + window_size_base)]
                Ans.append(rect)
            else:
                continue
    return Ans




# Making Partions from the list of possitive windows
def make_partition(positives):
    bins = []
    for rect in positives:
        found_bin = False
        for bin in bins: 
            if found_bin == False:
                for rect_bin in bin:
                    if(do_overlap_api(rect, rect_bin)):
                        bin.append(rect)
                        found_bin = True
                        break
            else:
                break
        if found_bin == False:
            bins.append([rect])
    return bins


def do_overlap_api(rectangles_1,rectangles_2):
    return do_overlap(rectangles_1[0], rectangles_1[1], rectangles_2[0], rectangles_2[1])


def do_overlap(l1, r1, l2, r2):
    # if rectangle has area 0, no overlap
    if l1.x == r1.x or l1.y == r1.y or r2.x == l2.x or l2.y == r2.y:
        return False
    # If one rectangle is on left side of other
    if l1.x > r2.x or l2.x > r1.x:
        return False
    # If one rectangle is above other
    if r1.y < l2.y or r2.y < l1.y:
        return False
    return True





# merging all windows in a particular partition
def get_all_boxes(partions):
    boxes = []
    for partion in partions:
        boxes.append(mean_rect(partion))
    return boxes


def mean_rect(rect_array):
    ls = [i[0] for i in rect_array]
    rs = [i[1] for i in rect_array]
    ls_av = mean_points(ls)
    rs_av = mean_points(rs)
    return (ls_av, rs_av)


def mean_points(l1_array):
    x_new = 0
    y_new = 0
    for i in range(len(l1_array)):
        x_new += l1_array[i].x
        y_new += l1_array[i].y
    x_new /= len(l1_array)
    y_new /= len(l1_array)
    return Point(int(x_new), int(y_new))







# Drwaing the boxes on the image
def draw_boxes(imageo, boxes):
    image = imageo.copy() 
    for box in boxes:
        image = cv2.rectangle(image, (box[0].x, box[0].y), (box[1].x, box[1].y), (0, 255, 0), 2)
    return image

In [6]:
window_size = 19
resize_shape = (19, 19)
ratio = 2